Descrição da aula: O objetivo da aula é explicar o que são variáveis dependentes e independentes, como os dados textuais (como nomes) se tornam valores numéricos. Para isso, uso a codificação one-hot (One-Hot Encoding). Finalmente, faço a separação entre conjunto de treino e conjunto de teste, de maneira automatizada.

In [1]:
# Biblioteca que possibilita acessar dataset e manipular os seus dados
import pandas as pd 

# biblioteca que possibilita realizar operações matémáticas com arrays
import numpy as np

Acessando o dataset com o pandas

In [2]:
# Quando os valores do dataset não estão sendo separados por virgula,
# devemos especificar no argumento delimiter, o tipo de operador que 
# esta realizando a separação dos dados.
base_de_dados = pd.read_csv('Dados/admission.csv', delimiter=';')

Visualizando a tabela com o head

In [3]:
base_de_dados.head(10)

,Name,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Approval
0,Lucas,337,118,4,4.5,4.5,9.65,1,1
1,Ana,324,107,4,4.0,4.5,8.87,1,1
2,Jose,316,104,3,3.0,3.5,8.00,1,1
3,Carlos,322,110,3,3.5,2.5,8.67,1,1
4,Zileide,314,103,2,2.0,3.0,8.21,0,0
5,Joana,330,115,5,4.5,3.0,9.34,1,1
6,Davi,321,109,3,3.0,4.0,8.20,1,1
7,Daniel,308,101,2,3.0,4.0,7.90,0,0
8,Marcelo,302,102,1,2.0,1.5,8.00,0,0


Selecionando apenas os valores do dataset

In [ ]:
# Acessando os valores de todas as colunas, exceto a ultima (coluna approval que queremos prever)
x = base_de_dados.iloc[:,:-1].values

# ira selecionar apenas os valores da ultima coluna (a coluna approval que queremos prever).
y = base_de_dados.iloc[:,-1].values

Preenchendo dados faltantes usando a classe SimpleImputer

In [5]:
# Import da classe SimpleImputer que tem como objetivo 
# preencher dados faltantes de uma base de dados.
from sklearn.impute import SimpleImputer

# Instância da classe SimpleImputer (criação do objeto) que irá conter
# os seguintes argumentos em seu construtor:
# missing_values: Irá conter o tipo do dado que esta faltando, no nosso
# caso serão os valores NaN (Not a Number)
# Strategy: Será a maneira como a classe irá preencher os dados faltantes,
# no nosso caso, vamos preencher com a mediana dos valores.
# Mediana: É o valor cuja posição na base de dados, dividi
# os valores em 50%
imputer = SimpleImputer(missing_values=np.nan, strategy='median')

# Ira inserir os valores da mediana nos dados faltantes.
# Observação: Como a primeira a coluna contém apenas os nomes
# dos alunos, iremos iniciar o preenchimento a partir da segunda 
# coluna
imputer = imputer.fit_transform(x[:,1:])

Transformando dados categóricos (textos) em rótulos numericos

In [ ]:
# Geralmente, os modelos de predição não aceitam colunas que possuem 
# valores categóricos. Dito isso, precisamos usar a classe LabelEncoder
# que tem como objetivo criar rótulos numéricos para variáveis categóricas
from sklearn.preprocessing import LabelEncoder

# Instância da classe labelencoder
labelencoder_x = LabelEncoder()

# Transformando os nomes (dados categóricos) em rótulos numéricos.
# Para realizar essa ação vamos usar o método fit_transform do label
# encoder que tem como objetivo gerar valores para os valores categóricos. A função recebe como argumento o conjunto de valores e a posição da coluna categórica que ganhará os rótulos numéricos.
x[:,0] = labelencoder_x.fit_transform(x[:, 0])

#  REMOVE a coluna 0 (que agora tem os rótulos numéricos) completamente e faz com que a coluna que antes era a 1 se torne a 0
x = x[:,1:]

# Ira criar os rótulos binários para a coluna categórica (já
#  com os rótulos numéricos). Dessa forma, os modelos de predição
# não irão utilizar os rótulos numéricos em cálculos. Para realizar 
# a criação dos rótulos binários vamos usar a função get_dummies
# do pandas que tem como objetivo, aplicar o one hoting na coluna
# especificada na função.
d = pd.get_dummies(x[:, 0])

# Agora vamos inserir as nova coluna no dataset original utilizando
# o insert da biblioteca numpy.
# x: Conjunto de dados que irá receber a nova coluna
# 0: Posição que a nova coluna ficará
# d.values: Valores binários da coluna inserida
# axis=1: Informa ao método a inserção de uma nova coluna
x = np.insert(x, 0, d.values, axis=1)

# Mostra os valores das colunas na tela
print(x)

[[False False False False False False False False True 337 118 4 4.5 4.5
  9.65 1]
 [False False False False False False False True False 324 107 4 4.0 4.5
  8.87 1]
 [False False False False True False False False False 316 104 3 3.0 3.5
  8.0 1]
 [False False True False False False False False False 322 110 3 3.5 2.5
  8.67 1]
 [False False False False False False True False False 314 103 2 2.0 3.0
  8.21 0]
 [False False False True False False False False False 330 115 5 4.5 3.0
  9.34 1]
 [False True False False False False False False False 321 109 3 3.0 4.0
  8.2 1]
 [False False False False False True False False False 308 101 2 3.0 4.0
  7.9 0]
 [True False False False False False False False False 302 102 1 2.0 1.5
  8.0 0]]


-> Separando os dados em treino e teste: Geralmente modelos de classificação e predição são separados em 2 categorias:

-> Treino: ira conter uma parte significativa da base de dados (geralmente 80% dos dados) e tem como objetivo treinar o modelo para ele descobrir padrões e a maneira como uma variável se relaciona com a outra.

-> Teste: Ira conter os valores reais da base de dados (geralmente 20% dos dados) e será utilizada como um gabarito para as classificações ou predições do modelo

-> Fazendo a mesma analogia que o professor, é como se os dados de treino 
fossem livros que um estudante (o modelo de predição/classificação) usa para aprender e o teste uma prova que verifica se o aluno aprendeu corretamente o 
conteúdo.

In [ ]:
# Import da função train_test_split da biblioteca sklearn.model
# que tem como objetivo separar os dados em treino e teste
from sklearn.model_selection import train_test_split

# Utilizando a função train_test_split: Para utilizar a função, vamos
# criar 4 variaveis que irão receber os valores gerados pelo train_split,
# as funções são:
# x_train: Ira conter os dados de treino do modelo
# x_test: Irá conter as caracteristicas necessárias para predição
# ou classificação do modelo.
# y_train: Ira conter os valores reais da base de dados
# y_test: Ira conter o gabarito que será comparado com os resultados
# gerados pelo modelo.
# x: Ira conter o conjunto de caracteristicas (entrada de x_test)
# y: será o alvo da classificação ou da predição
# test_size: Irá definir o tamanho do teste. No nosso caso, como definimos
# que o tamanho é 20% ele usara os 80% para treino
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

# Ira mostrar os dados utilizados como treino
print("DADOS DE TREINO")
print(x_train)




[[False False False True False False False False False 330 115 5 4.5 3.0
  9.34 1]
 [False False False False False False False False True 337 118 4 4.5 4.5
  9.65 1]
 [False False False False False True False False False 308 101 2 3.0 4.0
  7.9 0]
 [False False False False True False False False False 316 104 3 3.0 3.5
  8.0 1]
 [False False False False False False False True False 324 107 4 4.0 4.5
  8.87 1]
 [False False False False False False True False False 314 103 2 2.0 3.0
  8.21 0]
 [True False False False False False False False False 302 102 1 2.0 1.5
  8.0 0]]
